In [1]:
import random

from deap import base
from deap import creator
from deap import tools

from PIL import Image, ImageChops
import numpy as np
import math, operator

C:\Users\Joe\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [ ]:
# set filename
# open image
# convert image to format ready to be compared to



In [ ]:
# create evaluate functions

# create mutate function - mutate floats, how to stay in range??

# create crossover function

In [3]:
from deap import base, creator, tools, algorithms
from random import randint, random, gauss
from PIL import Image, ImageDraw
from functools import partial
from math import sqrt
import numpy


PIC = Image.open('template_big.jpg')
SIZE = 1000
NUMBER_OF_TRIANGLES = 50
POPULATION = 40
NGEN = 1000
POLY = 3


def gen_one_triangle():
    return (tuple([(randint(0, SIZE), randint(0, SIZE)) for i in range(POLY)]),
            randint(0,255), randint(0,255), randint(0,255), randint(0,30))
            #0, 0, 0, 0)


def triangles_to_image(triangles):
    im = Image.new('RGB', (SIZE, SIZE), (255, 255, 255))
    for tri in triangles:
        mask = Image.new('RGBA', (SIZE, SIZE))
        draw = ImageDraw.Draw(mask)
        draw.polygon(tri[0], fill=tri[1:])
        im.paste(mask, mask=mask)
        del mask, draw
    return im


def evaluate(im1, t2):
    im2 = triangles_to_image(t2)
    #pix1, pix2 = im1.load(), im2.load()
    #ans = 0
    #for i in range(SIZE):
    #    for j in range(SIZE):
    #        a1, a2, a3 = pix1[i, j]
    #        b1, b2, b3 = pix2[i, j]
    #        ans += (a1 - b1) ** 2 + (a2 - b2) ** 2 + (a3 - b3) ** 2
    #return 1 - (1. * sqrt(ans) / sqrt(SIZE * SIZE * 3 * 255 * 255)),

    h = ImageChops.difference(im1,im2).histogram()
    length = int(len(h)/3)

    r = h[length*0:length*1]
    g = h[length*1:length*2]
    b = h[length*2:length*3]

    h = [sum(x) for x in zip(r,g,b)]
    err = math.sqrt(sum(h*(i**2) for i, h in enumerate(h)) / float(SIZE * SIZE * 3)) / 256
    return 1 - err,



def mutate(triangles):
    e0 = evaluate(PIC, triangles)
    for i in range(2):
        tid = randint(0, NUMBER_OF_TRIANGLES - 1)
        oldt = triangles[tid]

        t = list(oldt)
        p = randint(0, 2 * POLY + 4 - 1)
        if p < 2 * POLY:
            points = list(t[0])
            pnt = list(points[int(p / 2)])
            #pnt[p%2] = max(0, min(SIZE, gauss(pnt[p%2], 10)))
            pnt[p % 2] = randint(0, SIZE)
            points[int(p / 2)] = tuple(pnt)
            t[0] = tuple(points)
        else:
            p -= 2 * POLY - 1
            #t[p] = max(0, min(255, int(gauss(t[p], 20))))
            t[p] = randint(0, 255)

        triangles[tid] = tuple(t)
        #if evaluate(PIC, triangles) > e0:
        #    break
        #else:
        #    triangles[tid] = oldt
    return triangles,

creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

toolbox = base.Toolbox()
toolbox.register("attr", gen_one_triangle)
toolbox.register("individual", tools.initRepeat,
                 creator.Individual, toolbox.attr, NUMBER_OF_TRIANGLES)
toolbox.register("population", tools.initRepeat,
                 list, toolbox.individual)

toolbox.register("evaluate", partial(evaluate, PIC))
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)


def main():
    pop = toolbox.population(n=POPULATION)
    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("std", numpy.std)
    stats.register("max", numpy.max)
    stats.register("avg", numpy.mean)
    stats.register("min", numpy.min)

    try:
        pop, log = algorithms.eaSimple(
            pop, toolbox, cxpb=0.5, mutpb=0.1, ngen=NGEN, stats=stats,
            halloffame=hof, verbose=True)
    finally:
        open('result.txt', 'w').write(repr(hof[0]))
        triangles_to_image(hof[0]).save('result.png')

if __name__ == '__main__':
    main()

IndexError: list index out of range

In [23]:
#import cProfile
#cProfile.run("main()")

gen	nevals	std      	max     	avg     	min     
0  	40    	0.0124998	0.512641	0.485019	0.464053
1  	19    	0.0102088	0.519625	0.496715	0.470821
2  	22    	0.00919675	0.530977	0.506478	0.488708
3  	25    	0.00957484	0.559689	0.515475	0.501053
4  	21    	0.00854375	0.559689	0.520422	0.507313
5  	27    	0.011595  	0.567538	0.526903	0.510604
6  	22    	0.0160027 	0.58467 	0.537019	0.515537
7  	18    	0.0178091 	0.58467 	0.550917	0.515117
8  	17    	0.00963565	0.591239	0.56743 	0.540628
9  	20    	0.00816968	0.590301	0.574304	0.562072
10 	21    	0.0080498 	0.593647	0.581754	0.558028
11 	24    	0.00533433	0.595184	0.586503	0.565864
12 	21    	0.00306378	0.595184	0.591339	0.581645
13 	24    	0.00476857	0.610848	0.59393 	0.57019 
14 	25    	0.00825048	0.611525	0.597446	0.566685
15 	26    	0.00810994	0.616748	0.60426 	0.591796
16 	19    	0.00631868	0.624264	0.610404	0.593601
17 	18    	0.00565479	0.62309 	0.613685	0.586252
18 	28    	0.00704593	0.629302	0.615959	0.580148
19 	26    	0.00330004	0